Aplicamos lo aprendido en las dos actividades en las que hicimos uso del dataset de DESI.
Es decir, lo que haremos es:
1.  Aplicar GridSearchCV para encontrar los mejores hiperparámetros de la red convolucional que elaboré como tarea y que clasifica espectros astronómicos (GALAXY, STAR, QSO).
2.  Encontrar los mejores hiperparámetros para predecir el redshif de los espectros.

1. Clasificacion espectral.